In [67]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [58]:
retail_df = pd.DataFrame()

In [59]:
def scrape(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [72]:
def get_product(url):
    soup = scrape(url)

    # Set up Chrome options to run in headless mode
    chrome_options = Options()
    chrome_options.add_argument('--headless')

    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'article')))

    product_elements = driver.find_elements(By.TAG_NAME, 'article')

    for product_element in product_elements:
        product_element.click()

        articles = soup.find_all('article', role='article')

        driver.back()

    driver.quit()

In [61]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [73]:
page_counter = 1
urls = ['https://www.auchan.fr/fruits-legumes/fruits/ca-n0301?page=1']

while page_counter <= get_max_page(urls[0]):
    get_product(urls[0][:-1] + str(page_counter))
    page_counter += 1
    break

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000104fcc4cc chromedriver + 4162764
1   chromedriver                        0x0000000104fc4654 chromedriver + 4130388
2   chromedriver                        0x0000000104c1bbc0 chromedriver + 293824
3   chromedriver                        0x0000000104c2a770 chromedriver + 354160
4   chromedriver                        0x0000000104c21af0 chromedriver + 318192
5   chromedriver                        0x0000000104c2017c chromedriver + 311676
6   chromedriver                        0x0000000104c2313c chromedriver + 323900
7   chromedriver                        0x0000000104c23278 chromedriver + 324216
8   chromedriver                        0x0000000104c64